In [ ]:
# import torch
# print(f"\n🔍 PyTorch: {torch.__version__}")
# print(f"🔍 GPU: {torch.cuda.is_available()}")
# print(f"🔍 Nome da GPU: {torch.cuda.get_device_name(0)}")

# # Nova forma de verificar ROCm a partir do PyTorch 2.0+
# print(f"🔍 Backend GPU: {'ROCm' if torch.version.hip else 'CUDA'}")
# print(f"🔍 Versão HIP: {torch.version.hip}\n")

In [7]:
from transformers import pipeline
import torch
import pdfplumber

In [8]:
# Verifica e configura o dispositivo (GPU/CPU)
device = 0 if torch.cuda.is_available() else -1  # 0 = GPU, -1 = CPU
print(f"🔧 Rodando em: {'GPU' if device == 0 else 'CPU'}")

🔧 Rodando em: GPU


In [10]:
# Carregar modelo BERTimbau pré-treinado para Q&A
qa_pipeline = pipeline(
    "question-answering",
    model="neuralmind/bert-base-portuguese-cased",
    tokenizer="neuralmind/bert-base-portuguese-cased",
    device = device
)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


In [11]:
# Exemplo com documento longo
with pdfplumber.open("arquivo.pdf") as pdf:
    documento = " ".join([page.extract_text() for page in pdf.pages])  # Seu documento aqui
pergunta = "Qual é o tema principal?"

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [12]:
# Dividir o documento em chunks (exemplo simplificado)
chunks = [documento[i:i+400] for i in range(0, len(documento), 300)]  # Overlap de 112 tokens

In [13]:
respostas = []
for chunk in chunks:
    resposta = qa_pipeline(question=pergunta, context=chunk)
    respostas.append(resposta)

In [14]:
# Selecionar a resposta com maior score
melhor_resposta = max(respostas, key=lambda x: x['score'])
print(melhor_resposta)

{'score': 0.0006417880649678409, 'start': 56, 'end': 62, 'answer': '-scale'}
